In [ ]:
class EventStudy():
    import pandas as pd
    import datetime
    from datetime import timedelta
    import pandas_datareader.data as web
    import fix_yahoo_finance as yf
    import matplotlib.pyplot as plt
    import numpy as np
    import statsmodels.api as sm
    def __init__(self,file_path):
        try:
            pd.read_csv(file_path)
        except TypeError:
            return f'Wrong File Type.'
        self.file = pd.read_csv(file_path)
        
    def datetime(self):
        self.dataframe = pd.DataFrame(self.file)
        self.company = self.dataframe['Ticker']
        self.year = self.dataframe['Year']
        self.month = self.dataframe['Month']
        self.day = self.dataframe['Day']
        self.market = self.dataframe['Market']
        self.industry = self.dataframe['Industry']
        
    def eventdate(self):
        eventdate = []
        for i in range(0,len(self.dataframe)):
            eventdate.append(datetime.datetime(self.year[i],self.month[i],self.day[i]))
        self.dataframe['EventDate'] = eventdate

    def sample_length(self):
        #suppose there is only one event in a month
        uniquedays = self.dataframe['EventDate'].unique()
        inter_days = []
        for i in range(0,len(uniquedays)):
            inter_days.append(int((uniquedays[i+1]-uniquedays[i])/np.timedelta64(1,'D')))
        self.sample_length = min(inter_days)
    
    def reg(self):
        eventdate_set = self.dataframe['EventDate'].unique()
        subdate_cumulative_abnormal_return = pd.DataFrame()
        for date in eventdate_set:
            subevent_df = self.dataframe[self.dataframe.EventDate == date]
            start_date = date - timedelta(days = self.sample_length)
            end_date = date + timedelta(days = self.sample_length)
            subevent_company_list = subevent_df['Ticker'].unique()
            subevent_cumulative_abnormal_return = pd.Dataframe()
            for company in subevent_campany_list:
                price = web.DataReader([company, subevent_df[subevent_df.Ticker == company]['Market']],'yahoo', start_date,end_date)
                close_price = price['Close']
                return_rate = close_price.pct_change()
#                 %matplotlib inline
#                 plt.scatter(return_rate[i],return_rate[m])
                x = return_rate[company]    
                X = sm.add_constant(x)
                y = return_rate[subevent_df[subevent_df.Ticker == company]['Market']]
                model = sm.OLS(y,X,missing='drop')
                result = model.fit()
#                 print(result.summary())
                (a,b) = result.params
#                 fig, ax = plt.subplots(figsize=(12,10))
#                 ax.plot(y)
#                 ax.plot(result.fittedvalues)
                return_rate['Abnormal_Return'] = y - x*b - a
                subevent_cumulative_abnormal_return[f'{company}'] = return_rate['AR'].cumsum()[1:]
            subevent_cumulative_abnormal_return['Average_CAR'] = subevent_cumulative_abnormal_return.apply(lamda x: x.sum()/len(subevent_company_list),axis = 1)
#                 fig, ax = plt.subplots(figsize=(12,10))
#                 ax.plot(rets['cumu'])
            subdate_cumulative_abnormal_return[f'{date}'] = subevent_cumulative_abnormal_return['Average_CAR']
        Average_CAR = subdate_cumulative_abnormal_return.apply(lamda x: x.sum()/len(eventdate_set))
    
    def visualization(self):
        pass

In [ ]:
instance_1 = EventStudy(file_path)
instance_1.reg()
instance_1.visualization()